In [1]:
import torch 
import pynvml
pynvml = pynvml
pynvml.nvmlInit()
nvml_preload = 0
nvml_prev = 0
pytorch_prev = 0
def nvml_used():
    handle = pynvml.nvmlDeviceGetHandleByIndex(torch.cuda.current_device())
    info   = pynvml.nvmlDeviceGetMemoryInfo(handle)
    return b2mb(info.used)
def b2mb(x): return int(x/2**20)
def consume_gpu_ram(n): return torch.ones((n, n)).cuda()
def consume_gpu_ram_64mb():  return consume_gpu_ram(2**12)
def consume_gpu_ram_256mb(): return consume_gpu_ram(2**13)
def mem(): 
    global nvml_preload, nvml_prev, pytorch_preload, pytorch_prev
    nvml_this = nvml_used()
    nvml_delta_cached = nvml_this - nvml_preload
    nvml_delta_used = nvml_this - nvml_prev
    nvml_prev = nvml_this
    
    pytorch_this = torch.cuda.memory_allocated()
    pytorch_delta_used = pytorch_this - pytorch_prev
    pytorch_prev = pytorch_this
    
    print(f"   nvml used: {nvml_delta_used:4d}, allocated: {nvml_delta_cached:4d}")
    print(f"pytorch used: {b2mb(pytorch_delta_used):4d}, allocated: {b2mb(torch.cuda.memory_cached()):4d}\n")

In [2]:
print("preloading:")
mem()
_ = torch.ones((1, 1)).cuda()
mem()
preload = nvml_used()
pytorch_preload = torch.cuda.memory_allocated()

print("\nrunning:")
x1 = consume_gpu_ram_64mb()
mem()
x2 = consume_gpu_ram_256mb()
mem()
del x2
mem()
x3 = consume_gpu_ram_64mb()
mem()

preloading:
   nvml used: 4779, allocated: 4779
pytorch used:    0, allocated:    0

   nvml used:  495, allocated: 5274
pytorch used:    0, allocated:    1


running:
   nvml used:   64, allocated: 5338
pytorch used:   64, allocated:   65

   nvml used:  256, allocated: 5594
pytorch used:  256, allocated:  321

   nvml used:    0, allocated: 5594
pytorch used: -256, allocated:  321

   nvml used:    0, allocated: 5594
pytorch used:   64, allocated:  321

